라이브러리

In [5]:
import numpy as np

1. ReLU 계층
    - ReLU(Rectified Linear Unit) 함수는 입력이 0보다 크면 그대로 출력하고, 0 이하면 0을 출력한다.



- 1.1 수식
    - $y = \begin{cases}x & (x > 0) \\0 & (x \leq 0)\end{cases}$
    - 역전파 시
        - $\frac{\partial y}{\partial x} = \begin{cases}1 & (x > 0) \\0 & (x \leq 0)\end{cases}$

- 1.2 ReLU 클래스 구현
    - 순전파: 0 이하인 값을 모두 0으로 만든다.
    - 역전파: 0 이하였던 위치의 gradient를 0으로 만든다.

In [1]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx


2. Sigmoid 계층
    - Sigmoid 함수는 입력을 0~1 범위로 부드럽게 압축한다.



- 2.1 수식
    - $y = \frac{1}{1 + e^{-x}}$
    - 역전파 시
        - $\frac{\partial y}{\partial x} = y(1 - y)$

- 2.2 Sigmoid 클래스 구현
    - 순전파: Sigmoid 변환 수행
    - 역전파: Sigmoid 출력 값을 이용해 gradient 계산

In [2]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx



3. Affine 계층
    - Affine 변환은 선형 변환(행렬곱) + 편향(bias) 덧셈을 의미한다.



- 3.1 수식
    - $Y=XW+B$

- 3.2 Affine 클래스 구현
    - 순전파 : 입력과 가중치의 행렬곱 + 편향 덧셈
    - 역전파 : 
        - 입력 x에 대한 기울기
        - 가중치 W에 대한 기울기
        - 편향 b에 대한 기울기

In [3]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx


4. Softmax-with-Loss 계층
    - Softmax 계층과 Cross Entropy Error 계층을 합친 것.
        - Softmax로 확률 분포를 만들고,
        - Cross Entropy로 Loss를 계산한다.


- 4.1 Softmax 함수
    - 입력을 안정적으로 정규화 (오버플로 방지)

In [4]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y


- 4.2 Cross Entropy Error 함수
    - y : 예측 확률
    - t : 정답 레이블 (원-핫 인코딩)

- 4.3 SoftmaxWithLoss 클래스 구현
    - 순전파: 소프트맥스 → 크로스 엔트로피 계산
    - 역전파: softmax 출력과 정답의 차이를 반환

In [11]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx


5. 테스트 예시

In [12]:
if __name__ == '__main__':
    swl = SoftmaxWithLoss()

    # 예시1: 거의 맞춘 경우
    a = np.array([1, 8, 3])
    t = np.array([0, 1, 0])

    print(swl.forward(a, t))  # 0.0076
    print(swl.backward())     # 작은 오차

    # 예시2: 많이 틀린 경우
    a = np.array([1, 3, 8])

    print(swl.forward(a, t))  # 5.0076
    print(swl.backward())     # 큰 오차


0.007620616629495912
[ 0.00030165 -0.00253058  0.00222893]
5.0076057626568575
[ 3.01653061e-04 -3.31104402e-01  3.30802749e-01]


| 계층 | 역할 | 순전파 | 역전파 |
|:----|:----|:------|:------|
| ReLU | 0 이하를 0으로 만든다 | 입력값 중 0 이하를 0으로 | 0 이하의 미분을 0으로 |
| Sigmoid | 0~1로 부드럽게 압축 | sigmoid 함수 적용 | y(1-y) 곱하기 |
| Affine | 선형 변환 | 행렬곱 + 편향 | 입력, 가중치, 편향에 대한 미분 계산 |
| Softmax-with-Loss | 확률화 + 손실 계산 | softmax + cross-entropy | 출력과 정답 차이 계산 |
